In [1]:
# Parameters
artificial_humans = "../../data/artificial_humans/ah_1_1/data/model.pt"
artificial_humans_model = "graph"
artificial_human_manager = "../../data/artificial_humans/ahm_1_0/data/model.pt"
artificial_human_manager_model = "graph"
output_path = "../../data/manager/vhm/ah_1_1__ahm_1_0"
n_episode_steps = 16

n_episodes = 100
labels = {}

In [2]:
%load_ext autoreload
%autoreload 2

import os
import torch as th
from aimanager.utils.utils import make_dir

from aimanager.manager.environment import ArtificialHumanEnv
from aimanager.manager.evaluation import ManagerEvaluator
from aimanager.artificial_humans import AH_MODELS
from aimanager.utils.array_to_df import using_multiindex, add_labels

output_path = os.path.join(output_path, 'data')

/home/mpib/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mpib/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
class ArtificialHumanManager(th.nn.Module):
    def __init__(self, model_name, model_path):
        super(ArtificialHumanManager, self).__init__()
        self.model = AH_MODELS[model_name].load(model_path)

    def get_action(self, state, edge_index, first):
        state = {k: v.unsqueeze(0).unsqueeze(-1) for k, v in state.items()}
        enc = self.model.encode(state, edge_index=edge_index, mask='manager_valid', y_encode=False)
        y_pred = self.model.predict_one(enc[0], reset_rnn=first, sample=True)[0]
        return y_pred.squeeze(-1)


In [15]:
device = th.device('cpu')
rec_device = th.device('cpu')
artifical_humans = AH_MODELS[artificial_humans_model].load(artificial_humans).to(device)
artifical_humans.device = device
manager = ArtificialHumanManager(model_name=artificial_human_manager_model, model_path=artificial_human_manager).to(device)
manager.model.device = device


env = ArtificialHumanEnv(
    artifical_humans=artifical_humans, n_agents=4, n_contributions=21, n_punishments=31, episode_steps=n_episode_steps, device=device)


evaluator = ManagerEvaluator(n_test_episodes=n_episodes, n_episode_steps=n_episode_steps, output_file=None)

evaluator.eval(manager, env)

recorder = evaluator.recorder

In [16]:
punishments = using_multiindex(recorder.memory['punishments'].numpy(), columns=['idx','round_number', 'agent'], value_name='punishments')
common_good = using_multiindex(recorder.memory['common_good'].numpy(), columns=['idx','round_number', 'agent'], value_name='common_good')
contributions = using_multiindex(recorder.memory['contributions'].numpy(), columns=['idx','round_number', 'agent'], value_name='contributions')
payoffs = using_multiindex(recorder.memory['payoffs'].numpy(), columns=['idx','round_number', 'agent'], value_name='payoffs')

df = punishments.merge(common_good).merge(contributions).merge(payoffs)

df = df.drop(columns=['idx'])

df = df.groupby(['round_number', 'agent']).mean().reset_index()
df = add_labels(df, labels=labels)



make_dir(output_path)
df.to_csv(os.path.join(output_path, 'trace.csv'))